# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [2]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

RANDOM_STATE = 110
ROOT_DIR = "data"

### 데이터 읽어오기


In [18]:
# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [19]:
THRESHOLD = len(train_data) / 2
train_data = train_data.dropna(thresh=THRESHOLD, axis=1)
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [20]:
l = ['Dam', 'Fill1', 'Fill2']

def selection(process):
    start_col = f'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_{process}'
    train_data[start_col] = train_data[start_col].apply(lambda x: np.nan if x == 'OK' else x)
    
    end_col = f'WorkMode Collect Result_{process}'
    
    selected_columns = train_data.loc[:, start_col:end_col]
    return selected_columns, start_col, end_col

def shifting(df, column):
    df_shifted = df.copy()
    nan_rows = df[df[column].isna()]
    
    for index in nan_rows.index:
        row = df_shifted.loc[index]
        shifted_row = row.shift(-1).fillna(pd.NA)
        df_shifted.loc[index] = shifted_row
    
    return df_shifted

for i in l:
    selected_columns_i, start_col_i, end_col_i = selection(i)
    df_shifted = shifting(selected_columns_i, start_col_i)
    train_data.loc[:, start_col_i:end_col_i] = df_shifted
    
train_data.to_csv('train_mod.csv', index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_21548\3244114554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[start_col] = train_data[start_col].apply(lambda x: np.nan if x == 'OK' else x)
C:\Users\USER\AppData\Local\Temp\ipykernel_21548\3244114554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[start_col] = train_data[start_col].apply(lambda x: np.nan if x == 'OK' else x)
C:\Users\USER\AppData\Local\Temp\ipykernel_21548\3244114554.py:5: SettingWithCopyWarning: 
A value is trying

In [7]:
train_data = pd.read_csv("train_mod.csv")
# train_data = pd.read_csv("shifted_train.csv")
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,114.612,19.9,7.0,127,1,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85.000,19.6,7.0,185,1,0.0,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,114.612,19.8,10.0,73,1,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85.000,19.9,12.0,268,1,0.0,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85.000,19.7,8.0,121,1,0.0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85.000,19.2,1.0,318,1,0.0,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,114.612,20.5,14.0,197,1,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85.000,19.7,1.0,27,1,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85.000,20.1,13.0,117,1,0.0,Normal


In [8]:
single_unique = [col for col in train_data.columns if train_data[col].nunique() == 1]
train_data = train_data.drop(single_unique, axis=1)

numeric_cols = train_data.select_dtypes(include=[int, float]).columns
concat_cols = numeric_cols.union(['target'])

train_data = train_data[concat_cols]
train_data

,1st Pressure 1st Pressure Unit Time_AutoClave,1st Pressure Collect Result_AutoClave,2nd Pressure Collect Result_AutoClave,2nd Pressure Unit Time_AutoClave,3rd Pressure Collect Result_AutoClave,3rd Pressure Unit Time_AutoClave,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Collect Result_Fill2,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Z Collect Result_Fill2,...,Stage3 Circle3 Distance Speed Collect Result_Dam,Stage3 Circle4 Distance Speed Collect Result_Dam,Stage3 Line1 Distance Speed Collect Result_Dam,Stage3 Line2 Distance Speed Collect Result_Dam,Stage3 Line3 Distance Speed Collect Result_Dam,Stage3 Line4 Distance Speed Collect Result_Dam,THICKNESS 1 Collect Result_Dam,THICKNESS 2 Collect Result_Dam,THICKNESS 3 Collect Result_Dam,target
0,240,0.312,0.493,1,0.499,120,240.0,240,2.5,33,...,5800,5800,5800,5800,5800,5800,0.000,0.000,0.000,Normal
1,241,0.311,0.311,121,0.498,121,240.0,240,2.5,33,...,4000,4000,4000,4000,4000,4000,0.000,0.000,0.000,Normal
2,241,0.315,0.490,1,0.498,121,1000.0,240,12.5,33,...,5800,5800,5800,5800,5800,5800,0.012,-0.022,0.003,Normal
3,241,0.299,0.300,121,0.500,121,1000.0,240,12.5,33,...,4000,4000,4000,4000,4000,4000,0.000,0.000,0.000,Normal
4,240,0.302,0.302,120,0.498,120,240.0,240,2.5,32,...,9000,9000,9000,9000,9000,9000,0.000,0.000,0.000,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,241,0.300,0.300,121,0.500,121,240.0,240,2.5,33,...,5000,5000,5000,5000,5000,5000,0.000,0.000,0.000,Normal
40502,240,0.305,0.493,1,0.498,120,1000.0,240,12.5,33,...,5800,5800,5800,5800,5800,5800,-0.019,-0.021,-0.118,Normal
40503,241,0.309,0.490,1,0.498,121,240.0,240,2.5,33,...,6000,6000,6000,6000,6000,6000,0.000,0.000,0.000,Normal
40504,241,0.302,0.303,121,0.499,121,1000.0,240,12.5,33,...,9000,9000,9000,9000,9000,9000,0.000,0.000,0.000,Normal


In [9]:
train_data.dropna()

,1st Pressure 1st Pressure Unit Time_AutoClave,1st Pressure Collect Result_AutoClave,2nd Pressure Collect Result_AutoClave,2nd Pressure Unit Time_AutoClave,3rd Pressure Collect Result_AutoClave,3rd Pressure Unit Time_AutoClave,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Collect Result_Fill2,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Z Collect Result_Fill2,...,Stage3 Circle3 Distance Speed Collect Result_Dam,Stage3 Circle4 Distance Speed Collect Result_Dam,Stage3 Line1 Distance Speed Collect Result_Dam,Stage3 Line2 Distance Speed Collect Result_Dam,Stage3 Line3 Distance Speed Collect Result_Dam,Stage3 Line4 Distance Speed Collect Result_Dam,THICKNESS 1 Collect Result_Dam,THICKNESS 2 Collect Result_Dam,THICKNESS 3 Collect Result_Dam,target
0,240,0.312,0.493,1,0.499,120,240.0,240,2.5,33,...,5800,5800,5800,5800,5800,5800,0.000,0.000,0.000,Normal
1,241,0.311,0.311,121,0.498,121,240.0,240,2.5,33,...,4000,4000,4000,4000,4000,4000,0.000,0.000,0.000,Normal
2,241,0.315,0.490,1,0.498,121,1000.0,240,12.5,33,...,5800,5800,5800,5800,5800,5800,0.012,-0.022,0.003,Normal
3,241,0.299,0.300,121,0.500,121,1000.0,240,12.5,33,...,4000,4000,4000,4000,4000,4000,0.000,0.000,0.000,Normal
4,240,0.302,0.302,120,0.498,120,240.0,240,2.5,32,...,9000,9000,9000,9000,9000,9000,0.000,0.000,0.000,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,241,0.300,0.300,121,0.500,121,240.0,240,2.5,33,...,5000,5000,5000,5000,5000,5000,0.000,0.000,0.000,Normal
40502,240,0.305,0.493,1,0.498,120,1000.0,240,12.5,33,...,5800,5800,5800,5800,5800,5800,-0.019,-0.021,-0.118,Normal
40503,241,0.309,0.490,1,0.498,121,240.0,240,2.5,33,...,6000,6000,6000,6000,6000,6000,0.000,0.000,0.000,Normal
40504,241,0.302,0.303,121,0.499,121,1000.0,240,12.5,33,...,9000,9000,9000,9000,9000,9000,0.000,0.000,0.000,Normal


In [10]:
# df_concat = train_data

### 언더 샘플링


데이타 불균형을 해결하기 위해 언더 샘플링을 진행합니다.


In [11]:
normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2350


target
AbNormal    2350
Normal      2350
Name: count, dtype: int64

### 데이터 분할


In [12]:
df_train, df_val = train_test_split(
    df_concat,
    test_size=0.2,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)

def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}" + f" ratio: {num_abnormal/num_normal}")

# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 1880, AbNormal: 1880 ratio: 1.0
  Total: Normal: 470, AbNormal: 470 ratio: 1.0


## 3. 모델 학습


### 모델 정의


In [13]:
model = RandomForestClassifier(random_state=RANDOM_STATE)

### 모델 학습


In [14]:
# features = []

# for col in df_train.columns:
#     try:
#         df_train[col] = df_train[col].astype(int)
#         features.append(col)
#     except:
#         continue

train_x = df_train.drop(columns=['target'])
train_y = df_train["target"]

model.fit(train_x, train_y)

RandomForestClassifier(random_state=110)

In [15]:
test_x = df_val.drop(columns=['target'])

test_y = df_val["target"]
pred_y = model.predict(test_x)

f1 = f1_score(test_y, pred_y, pos_label='Normal')

print(f"F1 Score: {f1:.4f}")

F1 Score: 0.5626


In [11]:
pred_y

array(['Normal', 'Normal', 'Normal', ..., 'Normal', 'Normal', 'Normal'],
      dtype=object)

## 4. 제출하기


### 테스트 데이터 예측


테스트 데이터 불러오기


In [53]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
THRESHOLD = len(test_data) / 2
test_data = test_data.dropna(thresh=THRESHOLD, axis=1)

l = ['Dam', 'Fill1', 'Fill2']

def selection(process):
    start_col = f'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_{process}'
    test_data[start_col] = test_data[start_col].apply(lambda x: np.nan if x == 'OK' else x)
    
    end_col = f'WorkMode Collect Result_{process}'
    
    selected_columns = test_data.loc[:, start_col:end_col]
    return selected_columns, start_col, end_col

def shifting(df, column):
    df_shifted = df.copy()
    nan_rows = df[df[column].isna()]
    
    for index in nan_rows.index:
        row = df_shifted.loc[index]
        shifted_row = row.shift(-1).fillna(pd.NA)
        df_shifted.loc[index] = shifted_row
    
    return df_shifted

for i in l:
    selected_columns_i, start_col_i, end_col_i = selection(i)
    df_shifted = shifting(selected_columns_i, start_col_i)
    test_data.loc[:, start_col_i:end_col_i] = df_shifted
    
test_data.to_csv('test_mod.csv', index=False)
test_data = pd.read_csv("test_mod.csv")

single_unique = [col for col in test_data.columns if test_data[col].nunique() == 1]
test_data = test_data.drop(single_unique, axis=1)

numeric_cols = test_data.select_dtypes(include=[int, float]).columns

test_data = test_data[numeric_cols]

model = RandomForestClassifier(random_state=RANDOM_STATE)

train_x = df_train.drop(columns=['target'])
train_y = df_train["target"]

common_cols = train_x.columns.intersection(test_data.columns)

train_x = train_x[common_cols]
test_x = test_data[common_cols]

model.fit(train_x, train_y)

pred_y = model.predict(test_x)

df_sub = pd.read_csv("submission.csv")
df_sub["target"] = pred_y

df_sub.to_csv("submission.csv", index=False)

ValueError: Length of values (40506) does not match length of index (17361)

In [ ]:
# df_test_x = test_data[features]

# for col in df_test_x.columns:
#     try:
#         df_test_x.loc[:, col] = df_test_x[col].astype(int)
#     except:
#         continue

In [ ]:
# test_pred = model.predict(df_test_x)
# test_pred

### 제출 파일 작성


In [ ]:
# # 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
# df_sub = pd.read_csv("submission.csv")
# df_sub["target"] = test_pred

# # 제출 파일 저장
# df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
